# import dbase3 - justpython3
---
## https://github.com/davidkwast/python-dbase3
---
#### https://davidkwast.github.io/slides/just-python-03-Brasil-SP-2019-11-09.html
---

## David Kwast

## https://twitter.com/kwast

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, '../../')
TEST_PATH = Path('../../tests/')

```python
import sys
import os.path
import struct
import datetime
import decimal
import io
import tempfile
```

In [2]:
import dbase3
dbase3

<module 'dbase3' from '../../dbase3.py'>

In [3]:
db = dbase3.DBase(TEST_PATH / 'dbf_files/dbase_03.dbf')
db

![](../images/dbf_libreoffice.png)

In [4]:
db[0]

Record 0

In [5]:
record = db[0]

In [6]:
type(record)

dbase3.Record

In [7]:
record._fields_data[0]

{'name': b'Point_ID', 'type': b'C', 'length': 12, 'decimal_count': 0}

In [8]:
record._values[0]

'0507121'

In [9]:
record.point_id

'0507121'

In [10]:
record._dict_data

{'point_id': '0507121',
 'type': 'CMP',
 'shape': 'circular',
 'circular_d': '12',
 'non_circul': '',
 'flow_prese': 'no',
 'condition': 'Good',
 'comments': '',
 'date_visit': datetime.date(2005, 7, 12),
 'time': '10:56:30am',
 'max_pdop': Decimal('5.2'),
 'max_hdop': Decimal('2.0'),
 'corr_type': 'Postprocessed Code',
 'rcvr_type': 'GeoXT',
 'gps_date': datetime.date(2005, 7, 12),
 'gps_time': '10:56:52am',
 'update_sta': 'New',
 'feat_name': 'Driveway',
 'datafile': '050712TR2819.cor',
 'unfilt_pos': 2,
 'filt_pos': 2,
 'data_dicti': 'MS4',
 'gps_week': 1331,
 'gps_second': Decimal('226625.000'),
 'gps_height': Decimal('1131.323'),
 'vert_prec': Decimal('3.1'),
 'horz_prec': Decimal('1.3'),
 'std_dev': Decimal('0.897088'),
 'northing': Decimal('557904.898'),
 'easting': Decimal('2212577.192')}

In [11]:
record.gps_date

datetime.date(2005, 7, 12)

In [12]:
record.gps_week

1331

In [13]:
record.std_dev

Decimal('0.897088')

```python
class DBase(object):
    '''Dbase doc'''
    
    def __init__(
        self, path_or_fd, indexes=None, encoding='ascii',
        dont_strip_fields=tuple(), cache_indexes=False, cache_indexes_path='',
    ):
        '''
        indexes: ({column: <column_name>, unique: <true_or_false>}, ...)
        '''
```
    ~ ~ ~ ~
```python
    # Header Parsing
    header = struct.unpack('<4BI2H20x', fd.read(32))
    meta = {}
    self.meta = meta
```
    ~ ~ ~ ~
```python
    # Header Parsing
    header = struct.unpack('<4BI2H20x', fd.read(32))
    meta = {}
    self.meta = meta
```
    ~ ~ ~ ~

    ~ ~ ~ ~
```python
    # Field Descriptor Array Parsing
    self.meta['fields'] = []
    self.meta['field_dict'] = {}

    # 0x0D stored as the Field Descriptor terminator
    while fd.read(1) != b'\x0D':
        fd.seek(fd.tell()-1)
        raw = struct.unpack('<11scI2B14x', fd.read(32))

        data = {}
        # field name in ASCII zero-filled
        field_name = raw[0].rstrip(b'\x00')
        data['name'] = field_name
        # field type in ASCII (C N L D or M)
        data['type'] = raw[1]
        # field data address, really necessary?
        #data['addr'] = raw[2]
        # field length in binary
        data['length'] = raw[3]
        # field decimal count in binary
        data['decimal_count'] = raw[4]

        if field_name not in self.meta['field_dict']:
            self.meta['fields'].append(data)
            self.meta['field_dict'][field_name] = data
```

In [14]:
for rec in db:
    print(rec.point_id, rec.condition, rec.gps_height)

0507121 Good 1131.323
0507122 Good 1125.142
0507123 Good 1127.570
0507125 Good 1125.364
05071210 Good 1118.605
05071216 Good 1117.390
05071217 Good 1125.714
05071219 Plugged 1110.786
05071224 Good 1077.924
05071225 Good 1082.990
05071229 Good 1096.860
05071231 Plugged 1105.113
05071232 Plugged 1101.939
05071236 Plugged 1125.517


In [15]:
results = db.find({'condition': 'Plugged'})

In [16]:
results

[Record 11, Record 12, Record 13, Record 7]

In [17]:
for rec in results:
    print(rec.point_id, rec.condition, rec.gps_height)

05071231 Plugged 1105.113
05071232 Plugged 1101.939
05071236 Plugged 1125.517
05071219 Plugged 1110.786


In [18]:
db.to_csv(TEST_PATH / 'export' / 'dbase_03.csv')

```csv
b'point_id',b'type',b'shape',b'circular_d',b'non_circul',b'flow_prese',b'condition',b'comments',b'date_visit',b'time',b'max_pdop',b'max_hdop',b'corr_type',b'rcvr_type',b'gps_date',b'gps_time',b'update_sta',b'feat_name',b'datafile',b'unfilt_pos',b'filt_pos',b'data_dicti',b'gps_week',b'gps_second',b'gps_height',b'vert_prec',b'horz_prec',b'std_dev',b'northing',b'easting'
b'0507121',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'10:56:30am',b'5.2',b'2.0',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'10:56:52am',b'New',b'Driveway',b'050712TR2819.cor',b'2',b'2',b'MS4',b'1331',b'226625.000',b'1131.323',b'3.1',b'1.3',b'0.897088',b'557904.898',b'2212577.192'
b'0507122',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'10:57:34am',b'4.9',b'2.0',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'10:57:37am',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'226670.000',b'1125.142',b'2.8',b'1.3',b'None',b'557997.831',b'2212576.868'
b'0507123',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'10:59:03am',b'5.4',b'4.4',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'10:59:12am',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'226765.000',b'1127.570',b'2.2',b'3.5',b'None',b'558184.757',b'2212571.349'
b'0507125',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'11:02:43am',b'3.4',b'1.5',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'11:03:12am',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'227005.000',b'1125.364',b'3.2',b'1.6',b'None',b'558703.723',b'2212562.547'
b'05071210',b'CMP',b'circular',b'15',b'',b'no',b'Good',b'',b'2005-07-12',b'11:15:20am',b'3.7',b'2.2',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'11:14:52am',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'227705.000',b'1118.605',b'1.8',b'2.1',b'None',b'558945.763',b'2212739.979'
b'05071216',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'12:13:23pm',b'4.4',b'1.8',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:13:57pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'231250.000',b'1117.390',b'3.1',b'1.2',b'None',b'559024.234',b'2212856.927'
b'05071217',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'12:16:46pm',b'4.4',b'1.8',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:17:12pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'231445.000',b'1125.714',b'3.2',b'1.3',b'None',b'559342.534',b'2213340.161'
b'05071219',b'CMP',b'circular',b'12',b'',b'no',b'Plugged',b'',b'2005-07-12',b'12:22:55pm',b'4.4',b'1.8',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:22:22pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'231755.000',b'1110.786',b'2.5',b'1.1',b'None',b'559578.776',b'2213560.247'
b'05071224',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'12:37:17pm',b'4.1',b'1.7',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:38:32pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'232725.000',b'1077.924',b'2.8',b'1.4',b'None',b'560582.575',b'2213759.022'
b'05071225',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'12:39:48pm',b'4.0',b'1.7',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:39:52pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'232805.000',b'1082.990',b'2.0',b'1.0',b'None',b'560678.501',b'2213716.657'
b'05071229',b'CMP',b'circular',b'12',b'',b'no',b'Good',b'',b'2005-07-12',b'12:49:05pm',b'3.7',b'1.7',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:49:07pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'233360.000',b'1096.860',b'2.4',b'1.2',b'None',b'560126.094',b'2213720.301'
b'05071231',b'CMP',b'circular',b'12',b'',b'no',b'Plugged',b'',b'2005-07-12',b'12:53:58pm',b'3.0',b'1.6',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:54:02pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'233655.000',b'1105.113',b'1.8',b'1.1',b'None',b'559952.331',b'2213689.001'
b'05071232',b'CMP',b'circular',b'12',b'',b'no',b'Plugged',b'',b'2005-07-12',b'12:55:47pm',b'3.5',b'1.7',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'12:55:47pm',b'New',b'Driveway',b'050712TR2819.cor',b'2',b'2',b'MS4',b'1331',b'233760.000',b'1101.939',b'2.1',b'1.1',b'1.223112',b'559870.352',b'2213661.918'
b'05071236',b'CMP',b'circular',b'12',b'',b'no',b'Plugged',b'',b'2005-07-12',b'01:08:40pm',b'3.3',b'1.6',b'Postprocessed Code',b'GeoXT',b'2005-07-12',b'01:08:42pm',b'New',b'Driveway',b'050712TR2819.cor',b'1',b'1',b'MS4',b'1331',b'234535.000',b'1125.517',b'1.8',b'1.2',b'None',b'559195.031',b'2213046.199'

```

In [19]:
db.to_sql(TEST_PATH / 'export' / 'dbase_03.sql', 'dbase03table')

```sql
BEGIN;

CREATE TABLE dbase03table (
	id INTEGER PRIMARY KEY,
	point_id TEXT,
	type TEXT,
	shape TEXT,
	circular_d TEXT,
	non_circul TEXT,
	flow_prese TEXT,
	condition TEXT,
	comments TEXT,
	date_visit DATE,
	time TEXT,
	max_pdop DECIMAL(10, 5),
	max_hdop DECIMAL(10, 5),
	corr_type TEXT,
	rcvr_type TEXT,
	gps_date DATE,
	gps_time TEXT,
	update_sta TEXT,
	feat_name TEXT,
	datafile TEXT,
	unfilt_pos DECIMAL(20, 10),
	filt_pos DECIMAL(20, 10),
	data_dicti TEXT,
	gps_week DECIMAL(12, 6),
	gps_second DECIMAL(24, 12),
	gps_height DECIMAL(32, 16),
	vert_prec DECIMAL(32, 16),
	horz_prec DECIMAL(32, 16),
	std_dev DECIMAL(32, 16),
	northing DECIMAL(32, 16),
	easting DECIMAL(32, 16)
);
```

```sql
INSERT INTO dbase03table VALUES (0,'0507121','CMP','circular','12','','no','Good','','2005-07-12','10:56:30am',5.2,2.0,'Postprocessed Code','GeoXT','2005-07-12','10:56:52am','New','Driveway','050712TR2819.cor',2,2,'MS4',1331,226625.000,1131.323,3.1,1.3,0.897088,557904.898,2212577.192);
INSERT INTO dbase03table VALUES (1,'0507122','CMP','circular','12','','no','Good','','2005-07-12','10:57:34am',4.9,2.0,'Postprocessed Code','GeoXT','2005-07-12','10:57:37am','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,226670.000,1125.142,2.8,1.3,NULL,557997.831,2212576.868);
INSERT INTO dbase03table VALUES (2,'0507123','CMP','circular','12','','no','Good','','2005-07-12','10:59:03am',5.4,4.4,'Postprocessed Code','GeoXT','2005-07-12','10:59:12am','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,226765.000,1127.570,2.2,3.5,NULL,558184.757,2212571.349);
INSERT INTO dbase03table VALUES (3,'0507125','CMP','circular','12','','no','Good','','2005-07-12','11:02:43am',3.4,1.5,'Postprocessed Code','GeoXT','2005-07-12','11:03:12am','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,227005.000,1125.364,3.2,1.6,NULL,558703.723,2212562.547);
INSERT INTO dbase03table VALUES (4,'05071210','CMP','circular','15','','no','Good','','2005-07-12','11:15:20am',3.7,2.2,'Postprocessed Code','GeoXT','2005-07-12','11:14:52am','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,227705.000,1118.605,1.8,2.1,NULL,558945.763,2212739.979);
INSERT INTO dbase03table VALUES (5,'05071216','CMP','circular','12','','no','Good','','2005-07-12','12:13:23pm',4.4,1.8,'Postprocessed Code','GeoXT','2005-07-12','12:13:57pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,231250.000,1117.390,3.1,1.2,NULL,559024.234,2212856.927);
INSERT INTO dbase03table VALUES (6,'05071217','CMP','circular','12','','no','Good','','2005-07-12','12:16:46pm',4.4,1.8,'Postprocessed Code','GeoXT','2005-07-12','12:17:12pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,231445.000,1125.714,3.2,1.3,NULL,559342.534,2213340.161);
INSERT INTO dbase03table VALUES (7,'05071219','CMP','circular','12','','no','Plugged','','2005-07-12','12:22:55pm',4.4,1.8,'Postprocessed Code','GeoXT','2005-07-12','12:22:22pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,231755.000,1110.786,2.5,1.1,NULL,559578.776,2213560.247);
INSERT INTO dbase03table VALUES (8,'05071224','CMP','circular','12','','no','Good','','2005-07-12','12:37:17pm',4.1,1.7,'Postprocessed Code','GeoXT','2005-07-12','12:38:32pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,232725.000,1077.924,2.8,1.4,NULL,560582.575,2213759.022);
INSERT INTO dbase03table VALUES (9,'05071225','CMP','circular','12','','no','Good','','2005-07-12','12:39:48pm',4.0,1.7,'Postprocessed Code','GeoXT','2005-07-12','12:39:52pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,232805.000,1082.990,2.0,1.0,NULL,560678.501,2213716.657);
INSERT INTO dbase03table VALUES (10,'05071229','CMP','circular','12','','no','Good','','2005-07-12','12:49:05pm',3.7,1.7,'Postprocessed Code','GeoXT','2005-07-12','12:49:07pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,233360.000,1096.860,2.4,1.2,NULL,560126.094,2213720.301);
INSERT INTO dbase03table VALUES (11,'05071231','CMP','circular','12','','no','Plugged','','2005-07-12','12:53:58pm',3.0,1.6,'Postprocessed Code','GeoXT','2005-07-12','12:54:02pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,233655.000,1105.113,1.8,1.1,NULL,559952.331,2213689.001);
INSERT INTO dbase03table VALUES (12,'05071232','CMP','circular','12','','no','Plugged','','2005-07-12','12:55:47pm',3.5,1.7,'Postprocessed Code','GeoXT','2005-07-12','12:55:47pm','New','Driveway','050712TR2819.cor',2,2,'MS4',1331,233760.000,1101.939,2.1,1.1,1.223112,559870.352,2213661.918);
INSERT INTO dbase03table VALUES (13,'05071236','CMP','circular','12','','no','Plugged','','2005-07-12','01:08:40pm',3.3,1.6,'Postprocessed Code','GeoXT','2005-07-12','01:08:42pm','New','Driveway','050712TR2819.cor',1,1,'MS4',1331,234535.000,1125.517,1.8,1.2,NULL,559195.031,2213046.199);

COMMIT;
```

```console
$ python dbase3.py 
Usage: dbase3.py [to_csv|to_sql]

$ python dbase3.py to_sql
Usage: dbase3.py < db file >  < db encoding >  < out file >

$ python dbase3.py to_csv
Usage: dbase3.py < db file >  < db encoding >  < out file >
```